In [44]:
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.stats.mstats import winsorize

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

## Opis kolumn
- id - identyfikator klienta/wniosku
- address_state - stan zamieszkania
- application_type - typ aplikacji **(zawiera tylko wartość INDIVIDUAL)**
- emp_length - długość zatrudnienia w latach
- emp_title - stanowisko prcay
- grade - ocena kredytowa **(A-G im wyższa tym lepiej)**
- home_ownership - status posiadania domu **(RENT, MORTAGE, OWN, OTHER, NONE)**
- issue_date - data wydania pożyczki
- last_credit_pull_date - data ostatniego sprawdzenia historii kredytowej
- last_payment_date - data ostatniej wpłaty na pożyczkę
- loan_status - status pożyczki **(Fully_Paid, Charged Off, Current)**
- next_payment_date - planowana data następnej wpłaty
- member_id - identyfikator klienta
- purpose - cel pożyczki **(13 kategorii plus other)**
- sub_grade - podkategoria oceny kredytowej **(bardziej dokładna niż grade, typu A1, A2, ...)**
- term - okres spłaty pożyczki **(36 months, 60 months)**
- verification_status - status weryfikacji dochodu klienta **(Not Verified, Verified, Source Verified)**
- annual_income - roczny dochód klienta
- dti (dent-to-income-ratio) - wskaźnik zadłużenia do dochodu
- installment - miesięczna rata
- int_rate - oprocentowanie
- loan_amount - kwota pożyczki
- total_acc - łączna liczba kont kredytowych klienta'
- total_payment - łączna suma spłat dokonanych przez klienta

### Wczytanie danych

In [45]:
data = pd.read_csv("loan_data.csv")

### Analiza emp_title

In [46]:
print("Unikalne emp_title:", data['emp_title'].nunique())
print("Braki emp_title:", data['emp_title'].isnull().sum())

Unikalne emp_title: 28525
Braki emp_title: 1438


In [47]:
data['is_employed'] = data['emp_title'].notnull().astype(int)
data.drop(columns=['emp_title'], inplace=True)

Ze względu na dużą różnorodność wykorzystujemy jedynie informację o tym czy osoba jest zatrudniona.

### Daty

In [48]:
data['last_credit_pull_date'] = pd.to_datetime(data['last_credit_pull_date'], errors='coerce')
data['issue_date'] = pd.to_datetime(data['issue_date'], errors='coerce')
data['date_diff'] = (data['last_credit_pull_date'] - data['issue_date']).dt.days
for c in ['last_credit_pull_date','issue_date','last_payment_date','next_payment_date']:
    if c in data.columns:
        data.drop(columns=[c], inplace=True)

C:\Users\alapr\AppData\Local\Temp\ipykernel_16900\2503067782.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['last_credit_pull_date'] = pd.to_datetime(data['last_credit_pull_date'], errors='coerce')


### Mapowanie zmiennych związanych z położeniem geograficznym
location, wealth, law_regulations

In [49]:
mapping = {
        'położenie': {
            'Northeast': ['CT','ME','MA','NH','RI','VT','NJ','NY','PA'],
            'Midwest':   ['IL','IN','MI','OH','WI','IA','KS','MN','MO','NE','ND','SD'],
            'South':     ['DE','DC','FL','GA','MD','NC','SC','VA','WV','AL','KY','MS','TN','AR','LA','OK','TX'],
            'West':      ['AZ','CO','ID','MT','NV','NM','UT','WY','AK','CA','HI','OR','WA']
        },
        'zamożność': {
            'high': ['DC','MD','MA','CA','CO','NJ','WA','AK','VA','NH','UT','HI','CT','MN','NY','RI','DE','NV','IL','TX','AZ','GA','WI','OR','ND','VT','FL','NE','NC','PA','SD','IA','MI','IN','OH','SC','KS','MO','TN','ME'],
            'med':  ['NM','ID','LA','OK','MT','KY','AL','WV','AR','MS'],
            'low':  ['WY','NM','MS','WV','AR','AL','KY','OK','LA','SC']
        },
        'prawo_kredytowe': {
            'restrictive': ['PA','NY','CA','AZ','HI','IN','IA','KS','MT','NE','NV','NH','NJ','NM','ND','OH','OK','OR','SD','TN','UT','WV','WI','WY'],
            'moderate':    ['DE','FL','GA','ID','IL','KY','ME','MD','MA','MI','MN','MO','NC','RI','TX','VT','VA','WA'],
            'lenient':     ['AK','AL','AR','CO','CT','DC','LA','MS','SC','WI','WY','SD']
        }
    }

In [50]:
loc_map = {st: region for region, states in mapping['położenie'].items() for st in states}
wealth_map = {st: grp for grp, states in mapping['zamożność'].items() for st in states}
law_map = {st: grp for grp, states in mapping['prawo_kredytowe'].items() for st in states}

In [51]:
if 'address_state' in data.columns:
    data['location'] = data['address_state'].map(loc_map)
    data['wealth'] = data['address_state'].map(wealth_map)
    data['law_regulations'] = data['address_state'].map(law_map)
    data.drop(columns=['address_state'], inplace=True)

In [52]:
### Redukcja ilości kategorii w zmiennej purpose

In [53]:
top2 = data['purpose'].value_counts().nlargest(2).index
data['purpose_grp'] = data['purpose'].where(data['purpose'].isin(top2), 'other')
data.drop(columns=['purpose'], inplace=True)

### Kolumna term

In [54]:
data['60_month_term'] = (data['term']=='60 months').astype(int)
data.drop(columns=['term'], inplace=True)

### Target

In [55]:
data['charged_off'] = (data['loan_status'] == 'Charged Off').astype(int)
X = data.drop(columns=['loan_status', 'charged_off'])
y = data['charged_off']

### Pipeline

In [56]:
d_num = X.select_dtypes(include=['int64','float64']).columns.tolist()
d_cat = X.select_dtypes(include=['object']).columns.tolist()
if 'emp_length' in d_num:
    d_num.remove('emp_length')

transformers = []
if d_num:
    transformers.append(('num', Pipeline([
        ('imp', SimpleImputer(strategy='median')),
        ('sc', StandardScaler())
    ]), d_num))
if d_cat:
    transformers.append(('cat', Pipeline([
        ('imp', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(handle_unknown='ignore', drop='first'))
    ]), d_cat))
if 'emp_length' in X.columns:
    transformers.append(('emp', Pipeline([
        ('imp', SimpleImputer(strategy='most_frequent')),
        ('ord', OrdinalEncoder(categories=[[
            '< 1 year','1 year','2 years','3 years','4 years','5 years',
            '6 years','7 years','8 years','9 years','10+ years'
        ]]))
    ]), ['emp_length']))

preprocessor = ColumnTransformer(transformers=transformers)

### Połączenie z targetem i zapis

In [57]:
X_proc = preprocessor.fit_transform(X)

if hasattr(X_proc, 'toarray'):
    df = pd.DataFrame(X_proc.toarray())
else:
    df = pd.DataFrame(X_proc)

df['charged_off'] = y.values
df.to_csv('loan_transformed.csv', index=False)